In [3]:
import numpy as np
import matplotlib.pyplot as plt
#from public_tests import *
#from utils import *

%matplotlib inline

In [4]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [6]:
print("First few elements of X_train:\n", X_train[:5])
print("Type of X_train:",type(X_train))
print("First few elements of y_train:", y_train[:5])
print("Type of y_train:",type(y_train))
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Type of X_train: <class 'numpy.ndarray'>
First few elements of y_train: [1 1 0 0 1]
Type of y_train: <class 'numpy.ndarray'>
The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


In [7]:

# Gcompute_entropy

def compute_entropy(y):
    """
    Computes the entropy for 
    
    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)
       
    Returns:
        entropy (float): Entropy at that node
        
    """
    # You need to return the following variables correctly
    entropy = 0.
    
    ### START CODE HERE ###
    lenth = y.size
    ct = 0
    for i in range(lenth):
        if y[i] == 1:
            ct += 1
    
    if ct == 0 or ct == lenth:
        return 0
    
    p1 = ct/lenth 
    entropy = -(p1*np.log2(p1) + (1-p1)*np.log2(1-p1))
    ### END CODE HERE ###        
    return entropy

In [9]:

# split_dataset

def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """
    
    # You need to return the following variables correctly
    left_indices = []
    right_indices = []
    length  = len(node_indices)
    ### START CODE HERE ###
    
    for i in range(length):
        if(X[node_indices[i],feature] == 1):
            left_indices.append(node_indices[i])
        else:
            right_indices.append(node_indices[i])
            
        
            
    ### END CODE HERE ###
        
    return left_indices, right_indices

In [10]:

# compute_information_gain

def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    ### START CODE HERE ###
    
    
    H_Pnode = compute_entropy(y_node)
    H_Pleft = compute_entropy(y_left)
    H_Pright = compute_entropy(y_right)
    
    
    Wleft = y_left.size/y_node.size
    Wright = y_right.size/y_node.size
    
    information_gain = H_Pnode - (Wleft*H_Pleft + Wright*H_Pright)

    ### END CODE HERE ###  
    
    return information_gain

In [11]:
#  get_best_split

def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    
    ### START CODE HERE ###
    max_info_gain = 0
    
    for i in range(num_features):
        info_gain = compute_information_gain(X, y, node_indices, feature=i)
        
        if(info_gain > max_info_gain):
            max_info_gain = info_gain
            best_feature = i
        
        
    ### END CODE HERE ##    
   
    return best_feature

In [12]:

tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [15]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)
//build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=3, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
-- Depth 2, Left: Split on feature: -1
   --- Left leaf node with indices [0, 1, 4, 7]
   --- Right leaf node with indices []
-- Depth 2, Right: Split on feature: -1
   --- Left leaf node with indices [5]
   --- Right leaf node with indices []
- Depth 1, Right: Split on feature: 1
-- Depth 2, Left: Split on feature: -1
   --- Left leaf node with indices []
   --- Right leaf node with indices [8]
-- Depth 2, Right: Split on feature: -1
   --- Left leaf node with indices []
   --- Right leaf node with indices [2, 3, 6, 9]
